# 🎫 Chapter 6 练习 1: 智能发券优化 - 把钱花在刀刃上

## 补贴的悖论

你是一家外卖平台的运营经理，手握 100 万张优惠券的预算。一个简单的问题：

> **发给谁？**

传统做法：「发给最可能下单的用户」—— 错！

为什么？因为最可能下单的用户**本来就会下单**，给他们发券只是**白送钱**！

正确做法：发给「**因为你发券才会下单**」的用户。这就是 **Uplift 建模**的核心！

---

## 🎯 学习目标

1. 理解用户分群策略（Persuadables, Sure Things, Lost Causes, Sleeping Dogs）
2. 掌握 Uplift 建模在发券场景的应用
3. 学习 ROI 优化决策方法

## 🌟 四类用户的直觉

想象你开了一家餐厅，要决定给谁发优惠券：

| 用户类型 | 不发券 | 发券 | Uplift | 策略 |
|----------|--------|------|--------|------|
| **Persuadables** (可说服者) | ❌ 不来 | ✅ 来了 | 高正 | 🎯 **重点发券** |
| **Sure Things** (必来者) | ✅ 来了 | ✅ 来了 | 0 | 💰 **浪费钱** |
| **Lost Causes** (流失者) | ❌ 不来 | ❌ 不来 | 0 | 🚫 **别浪费** |
| **Sleeping Dogs** (睡狗) | ✅ 来了 | ❌ 不来 | 负 | ⚠️ **千万别发** |

### Sleeping Dogs 是什么鬼？

有些用户收到促销信息反而会**产生反感**：
- 「这餐厅居然这么便宜，质量是不是有问题？」
- 「发这么多广告，太烦人了，取关！」
- 「居然用优惠券来「骗」我，以后不去了」

这类用户就像「睡着的狗」，千万别去打扰它！

## 📐 核心公式

### Uplift 定义

$$\text{Uplift}(x) = P(Y=1|T=1, X=x) - P(Y=1|T=0, X=x)$$

即：发券时的转化率 - 不发券时的转化率

### ROI 计算

$$\text{ROI} = \frac{\text{增量收入} - \text{成本}}{\text{成本}}$$

$$\text{增量收入} = \text{增量转化数} \times \text{每次转化收入}$$

$$\text{成本} = \text{发券数} \times \text{每张券成本}$$

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from sklearn.ensemble import GradientBoostingClassifier

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🎫")

## 练习 1.1: 生成营销场景数据

让我们模拟一个外卖平台的发券实验！

In [ ]:
def generate_simple_marketing_data(
    n_samples: int = 2000,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成简化的营销场景数据
    
    场景: 外卖平台发券实验
    - 特征: 用户年龄、历史订单数、最近活跃度
    - 处理: 是否发券 (T=1 发券, T=0 不发券)
    - 结果: 是否下单 (conversion=1 下单, conversion=0 未下单)
    
    用户类型:
    - Persuadables: 年轻 + 低频用户（被优惠券吸引）
    - Sure Things: 高频用户（本来就爱点外卖）
    - Sleeping Dogs: 低频老用户（反感促销）
    - Lost Causes: 其他
    """
    np.random.seed(seed)
    
    # 生成用户特征
    # age: 18-65 岁均匀分布
    age = np.random.uniform(18, 65, n_samples)
    
    # order_freq: 历史订单数，泊松分布 lambda=5
    order_freq = np.random.poisson(5, n_samples)
    
    # days_since_last: 距上次下单天数，指数分布 scale=10
    days_since_last = np.random.exponential(10, n_samples)
    
    # 随机分配处理 (50% 概率发券)
    T = np.random.binomial(1, 0.5, n_samples)
    
    # 确定用户类型
    user_type = []
    conversion = []
    
    for i in range(n_samples):
        # 根据规则判断用户类型
        # Persuadables: 年轻 (age < 30) + 低频 (order_freq < 5)
        # Sure Things: 高频 (order_freq >= 8)
        # Sleeping Dogs: 非常低频 (order_freq < 2) + 不年轻 (age >= 40)
        # Lost Causes: 其他
        
        if age[i] < 30 and order_freq[i] < 5:
            user_type.append('Persuadables')
        elif order_freq[i] >= 8:
            user_type.append('Sure Things')
        elif order_freq[i] < 2 and age[i] >= 40:
            user_type.append('Sleeping Dogs')
        else:
            user_type.append('Lost Causes')
        
        # 生成转化结果
        # 基线转化率: 0.1 + 0.02 * min(order_freq, 10)
        base_prob = 0.1 + 0.02 * min(order_freq[i], 10)
        
        # 根据用户类型调整转化率
        if user_type[i] == 'Persuadables':
            # 发券效果很好：+0.25
            prob = base_prob + (0.25 if T[i] == 1 else 0)
        elif user_type[i] == 'Sure Things':
            # 本来就会买：基线高，发券效果小
            prob = base_prob + 0.3 + (0.03 if T[i] == 1 else 0)
        elif user_type[i] == 'Sleeping Dogs':
            # 发券反而降低：-0.1
            prob = base_prob + 0.15 + (-0.1 if T[i] == 1 else 0)
        else:  # Lost Causes
            # 发不发都不买
            prob = base_prob + (0.01 if T[i] == 1 else 0)
        
        # 确保概率在 [0, 1] 范围内
        prob = np.clip(prob, 0, 1)
        conversion.append(np.random.binomial(1, prob))
    
    return pd.DataFrame({
        'age': age,
        'order_freq': order_freq,
        'days_since_last': days_since_last,
        'T': T,
        'conversion': conversion,
        'user_type': user_type
    })

In [ ]:
# 测试数据生成
df = generate_simple_marketing_data(n_samples=5000)

if df is not None and df['age'].iloc[0] is not None:
    print("营销数据生成成功! 🎉")
    print(f"\n样本量: {len(df)}")
    print(f"发券率: {df['T'].mean():.2%}")
    print(f"整体转化率: {df['conversion'].mean():.2%}")
    
    print(f"\n=== 用户类型分布 ===")
    print(df['user_type'].value_counts())
    
    # 计算各类型的真实 Uplift
    print(f"\n=== 各用户类型的真实 Uplift ===")
    for user_type in df['user_type'].unique():
        mask = df['user_type'] == user_type
        treat_conv = df.loc[mask & (df['T']==1), 'conversion'].mean()
        control_conv = df.loc[mask & (df['T']==0), 'conversion'].mean()
        uplift = treat_conv - control_conv
        print(f"  {user_type}: {uplift:+.3f}")
else:
    print("[TODO] 请完成 generate_simple_marketing_data 函数")

In [ ]:
# 可视化用户类型
if df is not None and df['age'].iloc[0] is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = {
        'Persuadables': '#2ecc71',
        'Sure Things': '#3498db',
        'Lost Causes': '#95a5a6',
        'Sleeping Dogs': '#e74c3c'
    }
    
    # 用户分布
    for user_type in df['user_type'].unique():
        mask = df['user_type'] == user_type
        axes[0].scatter(
            df.loc[mask, 'age'],
            df.loc[mask, 'order_freq'],
            alpha=0.5,
            c=colors.get(user_type, 'gray'),
            label=user_type,
            s=30
        )
    axes[0].set_xlabel('年龄')
    axes[0].set_ylabel('历史订单数')
    axes[0].set_title('用户类型在特征空间的分布')
    axes[0].legend()
    
    # Uplift 对比
    uplift_data = []
    for user_type in ['Persuadables', 'Sure Things', 'Lost Causes', 'Sleeping Dogs']:
        mask = df['user_type'] == user_type
        if mask.sum() > 0:
            treat_conv = df.loc[mask & (df['T']==1), 'conversion'].mean()
            control_conv = df.loc[mask & (df['T']==0), 'conversion'].mean()
            uplift_data.append({
                'type': user_type,
                'uplift': treat_conv - control_conv,
                'treat_conv': treat_conv,
                'control_conv': control_conv
            })
    
    if len(uplift_data) > 0:
        uplift_df = pd.DataFrame(uplift_data)
        x = range(len(uplift_df))
        
        bars = axes[1].bar(x, uplift_df['uplift'], 
                          color=[colors.get(t, 'gray') for t in uplift_df['type']])
        axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(uplift_df['type'], rotation=15)
        axes[1].set_ylabel('Uplift (转化率提升)')
        axes[1].set_title('各用户类型的真实 Uplift')
        
        # 添加数值标签
        for bar, val in zip(bars, uplift_df['uplift']):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:+.3f}', ha='center', va='bottom' if val > 0 else 'top')
    
    plt.tight_layout()
    plt.show()

## 练习 1.2: 训练 Uplift 模型

使用 T-Learner (Two-Model) 方法训练 Uplift 模型。

In [ ]:
class SimpleUpliftModel:
    """
    简单的 Uplift 模型 (T-Learner 方法)
    
    思想: 分别训练控制组和处理组的模型，差值即为 Uplift
    """
    
    def __init__(self):
        self.model_control = GradientBoostingClassifier(
            n_estimators=50, max_depth=4, random_state=42
        )
        self.model_treatment = GradientBoostingClassifier(
            n_estimators=50, max_depth=4, random_state=43
        )
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 Uplift 模型
        """
        # 分离控制组和处理组数据
        mask_control = (T == 0)
        mask_treatment = (T == 1)
        
        # 训练两个模型
        self.model_control.fit(X[mask_control], Y[mask_control])
        self.model_treatment.fit(X[mask_treatment], Y[mask_treatment])
        
        return self
    
    def predict_uplift(self, X: np.ndarray) -> np.ndarray:
        """
        预测 Uplift
        """
        # 分别预测两组的转化概率
        prob_treatment = self.model_treatment.predict_proba(X)[:, 1]
        prob_control = self.model_control.predict_proba(X)[:, 1]
        
        # Uplift = 处理组概率 - 控制组概率
        uplift = prob_treatment - prob_control
        
        return uplift

In [ ]:
# 训练 Uplift 模型
if df is not None and df['age'].iloc[0] is not None:
    X = df[['age', 'order_freq', 'days_since_last']].values
    T = df['T'].values
    Y = df['conversion'].values
    
    try:
        model = SimpleUpliftModel()
        model.fit(X, T, Y)
        uplift_pred = model.predict_uplift(X)
        
        if uplift_pred is not None:
            print("Uplift 模型训练成功! 🎉")
            print(f"\nUplift 预测统计:")
            print(f"  范围: [{uplift_pred.min():.3f}, {uplift_pred.max():.3f}]")
            print(f"  均值: {uplift_pred.mean():.3f}")
            print(f"  标准差: {uplift_pred.std():.3f}")
            
            # 与真实用户类型对比
            df['uplift_pred'] = uplift_pred
            print(f"\n各用户类型的预测 Uplift 均值:")
            for user_type in ['Persuadables', 'Sure Things', 'Lost Causes', 'Sleeping Dogs']:
                mean_uplift = df.loc[df['user_type']==user_type, 'uplift_pred'].mean()
                print(f"  {user_type}: {mean_uplift:+.3f}")
        else:
            print("[TODO] 请完成 predict_uplift 方法")
    except Exception as e:
        print(f"[TODO] 模型训练失败: {e}")

## 练习 1.3: 用户分群

根据 Uplift 预测分数将用户分群。

In [ ]:
def segment_users(
    df: pd.DataFrame,
    uplift_scores: np.ndarray,
    high_threshold: float = 0.1,
    low_threshold: float = 0.02
) -> pd.DataFrame:
    """
    根据 Uplift 分数将用户分群
    
    分群规则:
    - High Uplift: uplift >= high_threshold (建议发券)
    - Low Uplift: 0 < uplift < low_threshold (可发可不发)
    - Negative Uplift: uplift <= 0 (千万别发!)
    """
    df = df.copy()
    df['uplift_score'] = uplift_scores
    
    # TODO: 实现分群规则
    segments = []
    for i in range(len(df)):
        uplift = uplift_scores[i]
        
        # 你的代码: 根据 uplift 判断分群
        if uplift >= high_threshold:
            segment = 'High Uplift (Target!)'
        elif uplift >= low_threshold:
            segment = 'Medium Uplift'
        elif uplift > 0:
            segment = 'Low Uplift'
        else:
            segment = 'Negative Uplift (Avoid!)'
        
        segments.append(segment)
    
    df['segment'] = segments
    return df

In [ ]:
# 用户分群
if 'uplift_pred' in dir() and uplift_pred is not None:
    df_segmented = segment_users(df, uplift_pred)
    
    print("用户分群结果:")
    print("="*50)
    print(f"\n分群分布:")
    print(df_segmented['segment'].value_counts())
    
    print(f"\n各群组统计:")
    for segment in df_segmented['segment'].unique():
        mask = df_segmented['segment'] == segment
        avg_uplift = df_segmented.loc[mask, 'uplift_score'].mean()
        
        # 计算真实转化率
        treat_conv = df_segmented.loc[mask & (df_segmented['T']==1), 'conversion'].mean()
        control_conv = df_segmented.loc[mask & (df_segmented['T']==0), 'conversion'].mean()
        real_uplift = treat_conv - control_conv
        
        print(f"\n  {segment}:")
        print(f"    样本数: {mask.sum()}")
        print(f"    预测 Uplift: {avg_uplift:+.3f}")
        print(f"    真实 Uplift: {real_uplift:+.3f}")

## 练习 1.4: ROI 计算

计算不同发券策略的投资回报率。

In [ ]:
def calculate_roi_simple(
    df: pd.DataFrame,
    treatment_mask: np.ndarray,
    revenue_per_conversion: float = 100,
    cost_per_coupon: float = 15
) -> Dict[str, float]:
    """
    计算发券策略的 ROI
    
    Args:
        df: 数据
        treatment_mask: 哪些用户应该发券
        revenue_per_conversion: 每次转化的收入 (元)
        cost_per_coupon: 每张券的成本 (元)
    """
    # 发券的用户
    treated_users = df[treatment_mask == 1]
    
    # 在这些用户中，计算实际 uplift
    # (这里用随机分配的数据来估计)
    treat_conv = treated_users.loc[treated_users['T']==1, 'conversion'].mean()
    control_conv = treated_users.loc[treated_users['T']==0, 'conversion'].mean()
    
    if np.isnan(treat_conv) or np.isnan(control_conv):
        actual_uplift = 0
    else:
        actual_uplift = treat_conv - control_conv
    
    # 计算 ROI
    n_coupons = treatment_mask.sum()  # 发券数量
    expected_conversions = n_coupons * actual_uplift  # 预期增量转化数
    revenue = expected_conversions * revenue_per_conversion  # 收入
    cost = n_coupons * cost_per_coupon  # 成本
    roi = (revenue - cost) / cost if cost > 0 else 0  # ROI
    
    return {
        'roi': roi,
        'revenue': revenue,
        'cost': cost,
        'n_coupons': n_coupons,
        'expected_conversions': expected_conversions,
        'actual_uplift': actual_uplift
    }

In [ ]:
# 测试 ROI 计算
if 'uplift_pred' in dir() and uplift_pred is not None:
    # 策略: 给 Uplift > 0.05 的用户发券
    test_mask = (uplift_pred > 0.05).astype(int)
    
    roi_result = calculate_roi_simple(df, test_mask)
    
    if roi_result['roi'] is not None:
        print("ROI 计算结果:")
        print("="*40)
        print(f"  发券人数: {roi_result['n_coupons']}")
        print(f"  实际 Uplift: {roi_result['actual_uplift']:.3f}")
        print(f"  预期增量转化: {roi_result['expected_conversions']:.1f}")
        print(f"  成本: ¥{roi_result['cost']:.0f}")
        print(f"  收入: ¥{roi_result['revenue']:.0f}")
        print(f"  ROI: {roi_result['roi']:.2f}")
    else:
        print("[TODO] 请完成 calculate_roi_simple 函数")

## 练习 1.5: 策略对比

对比三种发券策略：
1. **Random**: 随机发券
2. **High Frequency**: 给高频用户发券（传统做法）
3. **Uplift Model**: 根据 Uplift 分数发券（因果推断）

In [ ]:
def compare_strategies(
    df: pd.DataFrame,
    uplift_scores: np.ndarray,
    budget_fraction: float = 0.3
) -> pd.DataFrame:
    """
    对比不同发券策略的效果
    """
    n_target = int(budget_fraction * len(df))
    results = []
    
    # TODO: 策略 1 - Random
    np.random.seed(42)
    random_idx = np.random.choice(len(df), n_target, replace=False)
    random_mask = np.zeros(len(df))
    random_mask[random_idx] = 1
    random_roi = calculate_roi_simple(df, random_mask)
    if random_roi['roi'] is not None:
        results.append({'strategy': 'Random', **random_roi})
    
    # TODO: 策略 2 - High Frequency
    # 选择 order_freq 最高的用户
    high_freq_idx = np.argsort(df['order_freq'].values)[-n_target:]
    high_freq_mask = np.zeros(len(df))
    high_freq_mask[high_freq_idx] = 1
    high_freq_roi = calculate_roi_simple(df, high_freq_mask)
    if high_freq_roi['roi'] is not None:
        results.append({'strategy': 'High Frequency', **high_freq_roi})
    
    # TODO: 策略 3 - Uplift Model
    # 选择 uplift_scores 最高的用户
    uplift_idx = np.argsort(uplift_scores)[-n_target:]
    uplift_mask = np.zeros(len(df))
    uplift_mask[uplift_idx] = 1
    uplift_roi = calculate_roi_simple(df, uplift_mask)
    if uplift_roi['roi'] is not None:
        results.append({'strategy': 'Uplift Model', **uplift_roi})
    
    return pd.DataFrame(results)

In [ ]:
# 策略对比
if 'uplift_pred' in dir() and uplift_pred is not None:
    print("=" * 60)
    print("策略对比实验")
    print("=" * 60)
    print(f"\n预算: 给 30% 的用户发券")
    
    comparison = compare_strategies(df, uplift_pred, budget_fraction=0.3)
    
    if comparison is not None and len(comparison) > 0:
        print(f"\nROI 对比:")
        display(comparison[['strategy', 'n_coupons', 'actual_uplift', 
                           'revenue', 'cost', 'roi']].round(2))
        
        # 找最佳策略
        best_idx = comparison['roi'].idxmax()
        best_strategy = comparison.loc[best_idx, 'strategy']
        best_roi = comparison.loc[best_idx, 'roi']
        
        print(f"\n最佳策略: {best_strategy}")
        print(f"最佳 ROI: {best_roi:.2f}")
        
        # 可视化
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        colors = ['#95a5a6', '#3498db', '#2ecc71']
        
        # ROI 对比
        bars = axes[0].bar(comparison['strategy'], comparison['roi'], color=colors)
        axes[0].axhline(y=0, color='red', linestyle='--')
        axes[0].set_ylabel('ROI')
        axes[0].set_title('不同策略的 ROI 对比')
        for bar, val in zip(bars, comparison['roi']):
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:.2f}', ha='center', va='bottom')
        
        # Uplift 对比
        bars = axes[1].bar(comparison['strategy'], comparison['actual_uplift'], color=colors)
        axes[1].axhline(y=0, color='red', linestyle='--')
        axes[1].set_ylabel('实际 Uplift')
        axes[1].set_title('不同策略的实际 Uplift 对比')
        for bar, val in zip(bars, comparison['actual_uplift']):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                        f'{val:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        print("\n关键洞察:")
        print("- Random: 随机发券，ROI 一般")
        print("- High Frequency: 给高频用户发券，可能ROI更低（浪费在Sure Things上）")
        print("- Uplift Model: 给可说服用户发券，ROI 最高！")

## 🤔 思考题

In [ ]:
# 思考题 1: 为什么 "Sure Things" 用户会造成补贴浪费?

answer_1 = """
你的答案:



"""
print(answer_1)

---

## 📝 思考题参考答案

### 思考题 1: 为什么 "Sure Things" 用户会造成补贴浪费?

**参考答案**:

Sure Things 是那些**无论你是否发券，都会购买的用户**。给他们发券的问题：

**1. 经济学解释**:
```python
Uplift = P(购买|发券) - P(购买|不发券)

对于 Sure Things:
P(购买|发券) = 0.9
P(购买|不发券) = 0.9
Uplift = 0  # 没有增量！

成本 = 券面额（如15元）
收益 = 0（他们本来就会买）
ROI = -100%  # 纯亏损！
```

**2. 补贴的目的是改变行为**:
```
发券的目的:
✅ 让不买的人买（Persuadables）
✅ 让买便宜商品的人买贵的（提升客单价）
❌ 给本来就会买的人打折（白送钱）

Sure Things:
行为不变 → 补贴无效 → 浪费预算
```

**3. 真实案例**:

```python
某外卖平台案例:

场景: 给所有用户发20元券

结果分析:
- 总用户: 10,000
- Sure Things (高频用户): 3,000 (30%)
- Persuadables (低频用户): 5,000 (50%)
- Lost Causes: 2,000 (20%)

成本:
给 Sure Things 发券成本 = 3,000 × 20 = 60,000元
这3000人本来就会点单（无券转化率90%）

实际收益:
增量订单 = 3,000 × (90% - 90%) = 0
净损失 = 60,000元（纯白送）

如果只给 Persuadables 发券:
成本 = 5,000 × 20 = 100,000元
转化率提升: 10% → 40%
增量订单 = 5,000 × 30% = 1,500
收入 = 1,500 × 40 (客单价) = 60,000元
ROI = (60,000 - 100,000) / 100,000 = -40%

虽然仍亏损，但比给 Sure Things 好得多！
（实际中可以调整券面额达到盈亏平衡）
```

**4. 识别 Sure Things 的方法**:

```python
方法1: 历史行为
if user.order_freq > 10:  # 高频用户
    likely_sure_thing = True

方法2: RFM 分析
if user.recency < 7天 and user.frequency > 8:
    likely_sure_thing = True

方法3: Uplift 建模
uplift_score = model.predict(user_features)
if uplift_score < 0.01:  # 极低 uplift
    likely_sure_thing = True
```

**5. 为什么传统营销会犯这个错误**:

传统营销看的是"响应率"（Response Rate）而非 Uplift:

```python
传统方法:
P(购买|特征) 高的用户 → 发券

问题:
- Sure Things 的 P(购买|特征) 本来就高
- 所以会被优先选中
- 但他们的 Uplift = 0！

正确方法:
Uplift = P(购买|T=1,X) - P(购买|T=0,X)
只给 Uplift 高的用户发券
```

**关键洞察**:
> **"会购买"不等于"因为券而购买"。Sure Things 造成补贴浪费的本质是：我们为一个不会改变的行为付费。**

---

### 思考题 2: 为什么 "Sleeping Dogs" 用户发券反而转化率下降？举一个实际例子。

**参考答案**:

Sleeping Dogs（沉睡的狗）指的是**发券后反而降低转化率的用户**。原因包括：

**1. 品牌认知负面影响**:

```python
用户心理:
"这个品牌居然要打折促销？"
→ "是不是质量不好/卖不出去？"
→ "我不想买便宜货"
→ 取消购买计划
```

真实案例:
```
奢侈品电商的教训:

某高端服装品牌:
- 对所有用户发30%折扣券
- 高端用户群体（客单价 > 5000元）:
  发券前转化率: 8%
  发券后转化率: 5%
  Uplift = -3% （负向！）

原因分析:
用户访谈发现：
"我买你家是因为品质和品牌，打折让我怀疑真伪"
"打折说明不值这个价，我去别的品牌看看"
```

**2. 促销疲劳**:

```python
用户历史:
- 之前从不发券
- 突然频繁发券

用户反应:
"既然总是有券，我等券再买"
→ 原本计划本周买的，拖到下周
→ 拖着拖着就不买了
```

真实案例:
```
电商平台的券促销陷阱:

初期:
不发券转化率: 3%

开始每周发券:
发券当天转化率: 5%（看起来很好！）
但非发券日转化率: 1%（用户学会了等券）

综合转化率:
(5% × 1天 + 1% × 6天) / 7 = 1.57%
反而比不发券时的 3% 更低！

Sleeping Dogs 效应:
部分用户完全停止非促销日购买
Uplift = -1.43%
```

**3. 信息过载 / 打扰成本**:

```python
场景: EDM / Push 频繁发券

用户反应:
"太烦了，每天都是促销信息"
→ 取关 / 拉黑 / 卸载 App
→ 失去这个客户
```

真实案例:
```
某 App 的 Push 灾难:

策略: 每天给低活用户推送优惠券

A/B 测试结果:
实验组（每天 Push）:
- 7天转化率: 2%
- 但 App 卸载率: 15%

对照组（不 Push）:
- 7天转化率: 1.5%
- App 卸载率: 2%

长期影响:
实验组流失大量用户（Sleeping Dogs）
长期 LTV 损失远超短期转化收益
```

**4. 价格锚定破坏**:

```python
心理机制:
用户建立了"正常价格"的心理锚点
频繁打折 → 锚点下移
下次不打折时觉得"太贵"

反常识现象:
发券后，未来不发券时的转化率永久下降
```

真实案例:
```
咖啡连锁的价格战后遗症:

某咖啡品牌:
- 长期定价: 中杯30元
- 某月推出: 8折优惠券

结果:
优惠月:
转化率: 15%（相比平时12%，+3%）

恢复原价后:
转化率: 8%（永久下降4%！）

原因:
用户认知: "30元太贵了，我等打折"
形成了 Sleeping Dogs 群体
```

**5. 识别 Sleeping Dogs 的特征**:

```python
人群特征:
- 品牌忠诚度高（在意品牌形象）
- 价格敏感度低（愿意付高价）
- 购买决策理性（不冲动消费）
- 对促销信息反感

数据信号:
- 历史客单价高
- 购买频次稳定（不受促销影响）
- 从未使用过优惠券
- EDM 打开率低/取消订阅

Uplift 模型识别:
uplift_score < -0.05  # 显著负向效应
```

**6. 如何应对 Sleeping Dogs**:

```python
策略1: 精准排除
if uplift_score < 0:
    exclude_from_campaign(user)

策略2: 差异化促销
高端用户: 不打折，送赠品/升级服务
普通用户: 价格促销

策略3: 控制频次
Max_promo_per_user_per_month = 1  # 避免疲劳

策略4: 长期 Holdout
永久保留 5% 用户作为对照
观察长期影响
```

**关键洞察**:
> **"不是所有营销都是好的营销。有时候，不打扰就是最好的策略。Sleeping Dogs 提醒我们：克制和精准比覆盖率更重要。"**

---

### 思考题 3: 在真实业务中，如何验证 Uplift 模型的准确性?

**参考答案**:

Uplift 模型的验证比普通预测模型复杂得多，因为我们**永远无法观测到同一个用户在两种处理下的结果**（反事实问题）。

**核心挑战**:
```python
观测到的数据:
- 如果用户在处理组: 看到 Y(1)，看不到 Y(0)
- 如果用户在对照组: 看到 Y(0)，看不到 Y(1)

Uplift = Y(1) - Y(0)  # 但我们永远看不到同一个用户的两个值！
```

**验证方法**:

---

**方法 1: Uplift Curve（最常用）**

```python
def plot_uplift_curve(y_true, treatment, uplift_scores):
    """
    Uplift Curve: 类似 ROC 曲线，但用于 Uplift
    
    思路:
    1. 按 uplift_score 从高到低排序用户
    2. 逐步扩大目标人群（top 10%, top 20%, ...）
    3. 计算每个阶段的实际 uplift
    4. 绘制曲线
    """
    # 按 uplift 分数排序
    sorted_idx = np.argsort(-uplift_scores)
    y_sorted = y_true[sorted_idx]
    t_sorted = treatment[sorted_idx]
    
    n = len(y_true)
    percentiles = np.arange(0.1, 1.01, 0.1)  # 10%, 20%, ..., 100%
    
    uplifts = []
    for p in percentiles:
        n_users = int(n * p)
        
        # 计算该人群的实际 uplift
        y_subset = y_sorted[:n_users]
        t_subset = t_sorted[:n_users]
        
        treated_conv = y_subset[t_subset == 1].mean()
        control_conv = y_subset[t_subset == 0].mean()
        uplift = treated_conv - control_conv
        
        uplifts.append(uplift)
    
    # 绘图
    plt.plot(percentiles * 100, uplifts, marker='o')
    plt.xlabel('Top X% 用户（按 Uplift 分数）')
    plt.ylabel('实际 Uplift')
    plt.title('Uplift Curve')
    plt.axhline(y=0, color='r', linestyle='--', label='基线')
    plt.grid(True)
    plt.show()
    
    return uplifts

# 理想情况:
# - Top 10% 用户的 Uplift 最高
# - 随着百分位增加，Uplift 递减
# - 证明模型有排序能力
```

**评估标准**:
```python
好的模型:
Top 10%: Uplift = 0.25
Top 30%: Uplift = 0.15
Top 50%: Uplift = 0.08
...
→ 递减趋势明显，模型有区分度

差的模型:
各百分位 Uplift 都差不多
→ 模型无用，等同随机
```

---

**方法 2: Qini Curve（更严格的指标）**

```python
def qini_coefficient(y, t, uplift_scores):
    """
    Qini 系数: 衡量 Uplift 模型的累积效果
    
    类似 AUC，但针对 Uplift
    """
    sorted_idx = np.argsort(-uplift_scores)
    y_sorted = y[sorted_idx]
    t_sorted = t[sorted_idx]
    
    n = len(y)
    n_treated = t.sum()
    n_control = n - n_treated
    
    qini_values = []
    for i in range(1, n+1):
        # 前 i 个用户中
        y_sub = y_sorted[:i]
        t_sub = t_sorted[:i]
        
        # 处理组和对照组的转化数
        treated_conversions = (y_sub * t_sub).sum()
        control_conversions = (y_sub * (1 - t_sub)).sum()
        
        # Qini 值 = 处理组转化数 - 对照组转化数 × (处理组人数/对照组人数)
        qini = treated_conversions - control_conversions * (n_treated / n_control)
        qini_values.append(qini)
    
    # Qini 系数 = 曲线下面积
    qini_auc = np.trapz(qini_values) / n
    
    return qini_auc, qini_values

# 解读:
# Qini AUC > 0: 模型有正向效果
# Qini AUC ≈ 0: 模型无用
# Qini AUC < 0: 模型有害（排序反了）
```

---

**方法 3: 分层 A/B 测试（金标准）**

```python
# 最严格的验证方法

步骤:
1. 用历史数据训练 Uplift 模型
2. 将模型预测分数分层:
   - High Uplift: 分数 > 0.15
   - Medium Uplift: 0.05 < 分数 ≤ 0.15
   - Low/Negative Uplift: 分数 ≤ 0.05

3. 在每一层内做 A/B 测试:
   for segment in [High, Medium, Low]:
       实验组: 发券
       对照组: 不发券
       
       测量实际 Uplift = 实验组转化率 - 对照组转化率

4. 对比预测 vs 实际:
   Segment        预测 Uplift   实际 Uplift   误差
   --------------|-------------|-------------|------
   High Uplift   | 0.25        | 0.22        | -12%
   Medium Uplift | 0.10        | 0.09        | -10%
   Low Uplift    | 0.02        | 0.01        | -50%

# 评估标准:
✅ High > Medium > Low（排序正确）
✅ 预测值与实际值接近（校准良好）
❌ 如果 Low 的实际 Uplift > High，模型失败
```

真实案例:
```python
某电商平台验证结果:

模型预测:
High Uplift 用户: 预测 Uplift = 0.20
Low Uplift 用户: 预测 Uplift = 0.02

A/B 测试验证:
High Uplift 用户:
- 实验组转化率: 15%
- 对照组转化率: 10%
- 实际 Uplift = 5% (绝对值) = 0.05 / 0.10 = 50%提升

咦？预测0.20，实际只有0.05？

问题:
模型输出的是"相对提升"还是"绝对提升"要搞清楚！

正确解读:
- 如果模型输出绝对提升: 预测0.20应该看到实际≈0.20
- 如果模型输出相对提升: 预测0.20 = 20%提升，
  10% × 1.20 = 12%，实际15%接近（考虑误差）

关键: 验证时要统一单位！
```

---

**方法 4: 财务指标验证（业务价值）**

```python
# 不仅要验证 Uplift 准确性，还要验证业务价值

验证维度:

1. ROI 验证:
   预期 ROI = (Uplift × 客单价 - 券成本) / 券成本
   
   实际运营:
   实际 ROI = (实际增量收入 - 券成本) / 券成本
   
   对比误差: |预期 ROI - 实际 ROI| / 实际 ROI

2. 增量成本 (Incremental Cost):
   有些用户会拿券买比预期更便宜的商品
   
   预期客单价: $50
   实际客单价: $45（用户策略性使用券）
   
   成本增加: $5 per order
   
   → 模型要考虑用户博弈行为

3. 长期影响:
   某些 Persuadables 被券吸引后，会变成 Sure Things
   
   首单 Uplift: 0.25（发券才买）
   第二单 Uplift: 0（已经是忠实用户）
   
   → 计算 LTV 而非单次 ROI
```

---

**方法 5: 模型诊断**

```python
# 检查模型是否学到了真实的异质性

诊断1: 特征重要性分析
feature_importance = uplift_model.get_feature_importance()

检查:
- 是否符合业务常识？
- 比如 "历史订单数" 对 Uplift 影响大 → 合理
- 比如 "用户ID" 对 Uplift 影响大 → 过拟合！

诊断2: Uplift 分布
uplift_scores = model.predict(X_test)

检查:
- 大部分用户 Uplift 是否为正？（应该混合）
- 是否有极端值？（> 1.0 或 < -1.0 不合理）
- 分布是否smooth？（太多尖刺 → 过拟合）

诊断3: 对照组验证
# 在对照组上，Uplift 分数应该与实际转化率无关

corr(uplift_scores[T==0], Y[T==0])

应该: ≈ 0（对照组不受处理影响）
如果: > 0.3 → 模型在拟合基线转化率，不是 Uplift！
```

---

**方法 6: Cross-validation（时间序列划分）**

```python
# Uplift 模型的交叉验证要小心

错误做法:
KFold(shuffle=True)  # 随机划分
→ 数据泄露！未来数据影响过去预测

正确做法:
TimeSeriesSplit  # 时间序列划分

# 确保:
# - 训练集: t < T
# - 测试集: t >= T
# - 模拟真实部署场景

validation_strategy = TimeSeriesSplit(n_splits=5)

for train_idx, test_idx in validation_strategy.split(X):
    model.fit(X[train_idx], T[train_idx], Y[train_idx])
    uplift_pred = model.predict(X[test_idx])
    
    # 在测试集上计算 Qini 系数
    qini = qini_coefficient(Y[test_idx], T[test_idx], uplift_pred)
    print(f"Fold Qini AUC: {qini}")

# 平均 Qini 系数 = 模型泛化能力
```

---

**完整验证 Checklist**:

```python
Uplift 模型验证清单:

□ 离线指标:
  □ Uplift Curve 递减
  □ Qini AUC > 0.1
  □ 特征重要性合理
  □ 无过拟合迹象

□ 小规模 A/B 测试:
  □ 分层测试（High/Medium/Low Uplift）
  □ 排序正确性验证
  □ 绝对值校准

□ 业务指标:
  □ ROI 符合预期（误差 < 20%）
  □ 考虑用户博弈行为
  □ 长期 LTV 影响

□ 持续监控:
  □ 每月重新验证
  □ 模型衰减检测
  □ 异常报警
```

**关键洞察**:
> **Uplift 模型验证的金标准是"分层A/B测试"。离线指标（Uplift Curve, Qini）只能作为初步筛选，真正的准确性必须通过实际业务验证。**

---

### 思考题 4: 如果预算有限 (比如只能发券给 20% 的用户)，应该如何调整发券策略?

**参考答案**:

预算约束下的发券策略需要在**覆盖率**和**精准度**之间权衡，核心是**最大化 ROI**。

---

**方法 1: 基于 Uplift 分数的 Top-K 选择**

```python
# 最简单直接的方法

def select_top_k(uplift_scores, budget_fraction=0.2):
    """
    选择 Uplift 分数最高的 top K% 用户
    """
    n = len(uplift_scores)
    k = int(n * budget_fraction)
    
    # 排序，选择 top k
    top_k_idx = np.argsort(-uplift_scores)[:k]
    
    return top_k_idx

# 使用:
top_20pct = select_top_k(uplift_scores, budget_fraction=0.2)
send_coupon(users[top_20pct])

# 优点: 简单，直接最大化预期 Uplift
# 缺点: 没考虑成本差异、约束条件
```

---

**方法 2: 成本-收益优化（考虑不同券面额）**

```python
# 如果可以给不同用户发不同面额的券

问题设定:
- 券类型: 5元, 10元, 15元
- 预算总额: 100万元
- 目标: ROI 最大化

建模:
decision_vars:
- x_i^5: 是否给用户 i 发 5元券
- x_i^10: 是否给用户 i 发 10元券
- x_i^15: 是否给用户 i 发 15元券

约束:
- x_i^5 + x_i^10 + x_i^15 ≤ 1  (每人最多一张券)
- Σ (5·x_i^5 + 10·x_i^10 + 15·x_i^15) ≤ 1,000,000  (预算)

目标:
max Σ_i Σ_c (uplift_i^c · revenue_i - cost_c) · x_i^c

其中:
- uplift_i^c: 用户 i 对券 c 的 Uplift
- revenue_i: 用户 i 的预期客单价
- cost_c: 券 c 的成本

求解:
from scipy.optimize import linprog

# 线性规划求解最优分配
result = linprog(
    c=-benefits,  # 负号因为 linprog 是最小化
    A_ub=constraints_matrix,
    b_ub=constraints_rhs,
    bounds=[(0, 1) for _ in range(n_vars)]
)

optimal_allocation = result.x
```

真实案例:
```python
某咖啡店的券分配策略:

用户细分:
1. 高频用户（30%）:
   - Uplift(5元券) = 0.05
   - Uplift(10元券) = 0.08
   - 预期客单价 = 50元

2. 低频用户（50%）:
   - Uplift(5元券) = 0.15
   - Uplift(10元券) = 0.28
   - Uplift(15元券) = 0.35
   - 预期客单价 = 35元

预算: 100万，只能覆盖 20% 用户

策略 A (均等):
每人发 10元券
覆盖: 100,000 用户
成本: 100万
预期收益: ...

策略 B (差异化):
- 高频用户: 发 5元券（性价比高）
- 低频用户: 发 15元券（Uplift 大）
覆盖: 120,000 用户（因为节省了给高频用户的钱）
成本: 100万
预期收益: 提升 25%！

结论: 差异化券面额 > 统一券
```

---

**方法 3: 约束优化（考虑业务规则）**

```python
# 实际业务中有很多约束

常见约束:

1. 最小覆盖约束:
   每个用户群体至少覆盖 X%
   
   例如:
   - 新用户必须至少覆盖 10%（战略考虑）
   - 老用户可以只覆盖 5%

2. 分散化约束:
   不要把所有券集中在某个细分市场
   
   例如:
   - 单个渠道获取的用户占比 < 30%

3. 时间分散:
   不要一次性发完所有券
   
   策略:
   - 第1周: 发 30%
   - 第2周: 根据第1周效果调整，发 40%
   - 第3周: 发剩余 30%

实现:
def constrained_allocation(
    uplift_scores,
    user_segments,  # 用户所属细分市场
    budget_fraction=0.2,
    min_coverage_per_segment=0.05
):
    n_users = len(uplift_scores)
    k_total = int(n_users * budget_fraction)
    
    selected = []
    
    # 先满足最小覆盖约束
    for segment in np.unique(user_segments):
        segment_mask = (user_segments == segment)
        n_segment = segment_mask.sum()
        min_k = int(n_segment * min_coverage_per_segment)
        
        # 在该细分市场内选择 Uplift 最高的 min_k 用户
        segment_scores = uplift_scores[segment_mask]
        segment_idx = np.where(segment_mask)[0]
        top_in_segment = segment_idx[np.argsort(-segment_scores)[:min_k]]
        
        selected.extend(top_in_segment)
    
    # 剩余预算用于全局最优
    remaining = k_total - len(selected)
    if remaining > 0:
        remaining_mask = np.ones(n_users, dtype=bool)
        remaining_mask[selected] = False
        
        remaining_scores = uplift_scores[remaining_mask]
        remaining_idx = np.where(remaining_mask)[0]
        top_remaining = remaining_idx[np.argsort(-remaining_scores)[:remaining]]
        
        selected.extend(top_remaining)
    
    return np.array(selected)
```

---

**方法 4: 动态分配（分批发放 + 在线学习）**

```python
# 不要一次性用完预算，边发边学

多臂老虎机 (Multi-Armed Bandit) 方法:

算法: Thompson Sampling

初始:
- 将用户分为 K 个群体（基于特征聚类）
- 每个群体的 Uplift 先验分布: Beta(α=1, β=1)

每轮迭代:
1. 从每个群体的 Beta 分布采样 Uplift 估计:
   uplift_k ~ Beta(α_k, β_k)

2. 计算每个群体的"期望收益":
   value_k = uplift_k × revenue_k - cost_k

3. 选择 value_k 最高的群体，发券给其中一部分用户

4. 观测实际转化情况，更新 Beta 分布:
   if 用户转化:
       处理组: α_k += 1
       对照组: β_k += 1
   else:
       处理组: β_k += 1
       对照组: α_k += 1

5. 重复，直到预算用完

优点:
- 自动平衡探索（Exploration）和利用（Exploitation）
- 对初始模型不准确有鲁棒性
- 实时优化

代码实现:
class BanditCouponAllocator:
    def __init__(self, n_segments):
        self.alpha = np.ones(n_segments)
        self.beta = np.ones(n_segments)
    
    def select_segment(self):
        # Thompson Sampling
        sampled_uplifts = np.random.beta(self.alpha, self.beta)
        return np.argmax(sampled_uplifts)
    
    def update(self, segment, treated, converted):
        if treated:
            if converted:
                self.alpha[segment] += 1
            else:
                self.beta[segment] += 1
        else:
            # 对照组观测（假设有 Holdout）
            if converted:
                self.beta[segment] += 1
            else:
                self.alpha[segment] += 1
    
    def allocate(self, budget, users_per_round=100):
        rounds = budget // users_per_round
        
        for r in range(rounds):
            segment = self.select_segment()
            # 给该 segment 发券...
            # 观测结果，更新分布
```

---

**方法 5: 敏感性分析（鲁棒性考虑）**

```python
# Uplift 模型有不确定性，要考虑

场景: 模型预测 Uplift = 0.20，但置信区间 [0.10, 0.30]

保守策略:
使用置信下界做决策:
uplift_conservative = uplift_pred - 1.96 × uplift_std

选择用户:
selected = users[uplift_conservative > threshold]

理由:
- 即使在最坏情况下，也能保证正 ROI
- 避免过度自信

代码:
def robust_allocation(uplift_mean, uplift_std, budget_fraction=0.2):
    # 使用 95% 置信下界
    uplift_lower = uplift_mean - 1.96 * uplift_std
    
    # 只选择下界仍然为正的用户
    positive_uplift = uplift_lower > 0.05
    
    # 在这些用户中选 top K
    candidate_scores = uplift_lower[positive_uplift]
    candidate_idx = np.where(positive_uplift)[0]
    
    k = int(len(uplift_mean) * budget_fraction)
    selected = candidate_idx[np.argsort(-candidate_scores)[:k]]
    
    return selected
```

---

**完整决策框架**:

```python
预算有限时的发券策略（20% 覆盖率）:

步骤 1: 初步筛选
□ 排除 Sleeping Dogs（Uplift < 0）
□ 排除 ROI 为负的用户（Uplift × Revenue < Cost）

步骤 2: 分层分配
□ 为不同细分市场设置最小覆盖（战略考虑）
□ 在细分市场内按 Uplift 排序

步骤 3: 券面额优化
□ 高 Uplift 但低客单价 → 小额券
□ 高 Uplift 且高客单价 → 大额券
□ 最大化 (Uplift × Revenue - Cost)

步骤 4: 时间分散
□ 第1批 (40%预算): 高置信度用户
□ 第2批 (40%预算): 根据第1批效果调整
□ 第3批 (20%预算): 探索 + 剩余预算

步骤 5: 在线优化
□ 使用 Bandit 算法动态调整
□ 每周重新评估，淘汰低效群体

步骤 6: 监控 & 调整
□ 每日监控 ROI
□ 如果 ROI < 0，立即停止该群体
□ 将预算转移到高 ROI 群体
```

**关键洞察**:
> **预算有限时，要从"覆盖率最大化"转向"ROI最大化"。宁可只给1%的用户发券但ROI很高，也不要为了覆盖率而浪费预算在低Uplift用户上。**

---

### 思考题 5: Uplift 建模相比传统的 "响应率建模" 有什么本质区别?

**参考答案**:

这是理解 Uplift 建模的核心问题。很多公司仍在用响应率模型做营销决策，导致严重的资源浪费。

---

**本质区别总结**:

| 维度 | 响应率建模 | Uplift 建模 |
|------|------------|-------------|
| **目标变量** | $P(Y=1\|X)$ | $\tau(X) = P(Y=1\|T=1,X) - P(Y=1\|T=0,X)$ |
| **回答问题** | 谁会购买？ | 谁因为干预而购买？ |
| **训练数据** | 只需要结果数据 | 需要实验数据（T, X, Y） |
| **反事实** | ❌ 不考虑 | ✅ 核心 |
| **因果性** | 相关性 | 因果性 |
| **业务价值** | 会误导决策 | 直接优化 ROI |

---

**详细分析**:

**1. 建模目标不同**:

```python
响应率模型:
预测: P(购买 | 用户特征)

例如:
model = LogisticRegression()
model.fit(X, Y)
prob = model.predict_proba(X_test)[:, 1]

# 给 prob 最高的用户发券
selected = users[prob > 0.5]

问题:
prob 高的用户 = 本来就会买的用户（Sure Things）
→ 发券给他们是浪费！

---

Uplift 模型:
预测: P(购买|发券, X) - P(购买|不发券, X)

例如:
uplift_model = TLearner()
uplift_model.fit(X, T, Y)
uplift = uplift_model.predict(X_test)

# 给 uplift 最高的用户发券
selected = users[uplift > 0.1]

结果:
uplift 高的用户 = 会被券影响的用户（Persuadables）
→ ROI 最大化！
```

**示例对比**:

```python
场景: 外卖平台发20元券

用户 A:
- 响应率: P(购买) = 90%（高频用户）
- Uplift: P(购买|券) - P(购买|无券) = 91% - 90% = 1%

用户 B:
- 响应率: P(购买) = 30%（低频用户）
- Uplift: P(购买|券) - P(购买|无券) = 55% - 30% = 25%

响应率模型的决策:
选择 A（响应率 90% > 30%）

结果:
- 成本: 20元
- 增量订单: 1% 概率
- 预期增量收入: 0.01 × 40元 = 0.4元
- ROI: (0.4 - 20) / 20 = -98% 💸

Uplift 模型的决策:
选择 B（Uplift 25% > 1%）

结果:
- 成本: 20元
- 增量订单: 25% 概率
- 预期增量收入: 0.25 × 40元 = 10元
- ROI: (10 - 20) / 20 = -50%（仍亏损，但比 A 好得多）

如果客单价更高或券面额更低，B 可能正 ROI！
```

---

**2. 训练数据需求不同**:

```python
响应率模型:
数据需求:
user_id | features | purchased
--------|----------|----------
001     | [...]    | 1
002     | [...]    | 0

# 普通监督学习，不需要实验

---

Uplift 模型:
数据需求:
user_id | features | treatment | purchased
--------|----------|-----------|----------
001     | [...]    | 1         | 1
002     | [...]    | 0         | 0

# 必须有对照组！需要实验（A/B test）

为什么需要对照组？
- 要估计 Uplift = P(Y|T=1,X) - P(Y|T=0,X)
- 需要同时观测处理组和对照组
- 不做实验，无法估计反事实 P(Y|T=0,X)
```

**如何获取 Uplift 训练数据**:

```python
方法 1: 随机实验
# 最理想
randomly_assign_treatment(users, prob=0.5)

方法 2: 回溯历史实验
# 如果之前做过 A/B test
historical_experiment_data = load_past_ab_tests()

方法 3: Quasi-experimental
# 利用自然实验
# 例如: 系统故障导致部分用户未收到券（意外对照组）
```

---

**3. 对 Sure Things 的处理不同**:

```python
四类用户:

1. Sure Things:
   - P(Y=1|T=1,X) = 0.9
   - P(Y=1|T=0,X) = 0.9
   - Uplift = 0

2. Persuadables:
   - P(Y=1|T=1,X) = 0.6
   - P(Y=1|T=0,X) = 0.1
   - Uplift = 0.5

3. Lost Causes:
   - P(Y=1|T=1,X) = 0.05
   - P(Y=1|T=0,X) = 0.05
   - Uplift = 0

4. Sleeping Dogs:
   - P(Y=1|T=1,X) = 0.3
   - P(Y=1|T=0,X) = 0.5
   - Uplift = -0.2

响应率模型的排序:
1. Sure Things (P=0.9) ← 优先选择
2. Persuadables (P=0.6 when T=1)
3. Sleeping Dogs (P=0.5 when T=0)
4. Lost Causes (P=0.05)

→ 给 Sure Things 发券 → 浪费！

Uplift 模型的排序:
1. Persuadables (Uplift=0.5) ← 优先选择 ✅
2. Sure Things (Uplift=0)
3. Lost Causes (Uplift=0)
4. Sleeping Dogs (Uplift=-0.2) ← 绝不选择 ✅

→ 精准定向，ROI 最大化
```

---

**4. 模型复杂度不同**:

```python
响应率模型:
# 单模型
model = SomeClassifier()
model.fit(X, Y)

---

Uplift 模型:
# 通常需要多个模型

方法 1: T-Learner (Two Models)
model_control = Classifier()
model_treatment = Classifier()

model_control.fit(X[T==0], Y[T==0])
model_treatment.fit(X[T==1], Y[T==1])

uplift = model_treatment.predict(X) - model_control.predict(X)

方法 2: X-Learner (Four Models)
# Stage 1: 响应函数
model_0 = fit(X[T==0], Y[T==0])
model_1 = fit(X[T==1], Y[T==1])

# Stage 2: 伪 Uplift
D_0 = model_1.predict(X[T==0]) - Y[T==0]
D_1 = Y[T==1] - model_0.predict(X[T==1])

tau_0 = fit(X[T==0], D_0)
tau_1 = fit(X[T==1], D_1)

# Stage 3: 加权
propensity = fit_propensity(X, T)
uplift = propensity * tau_0 + (1-propensity) * tau_1

方法 3: S-Learner (Single Model with Treatment as Feature)
model = Classifier()
X_with_T = np.hstack([X, T.reshape(-1, 1)])
model.fit(X_with_T, Y)

# 预测 Uplift
X_t1 = np.hstack([X, np.ones((n, 1))])
X_t0 = np.hstack([X, np.zeros((n, 1))])
uplift = model.predict(X_t1) - model.predict(X_t0)
```

---

**5. 评估指标不同**:

```python
响应率模型:
指标: AUC, Precision, Recall

# 评估"预测购买概率"的准确性

from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_true, y_pred_proba)

---

Uplift 模型:
指标: Qini Coefficient, Uplift Curve

# 评估"区分 Persuadables 能力"

def qini_coefficient(y, t, uplift_scores):
    # ... (见前面思考题 3)
    return qini_auc

# 或者直接看业务指标
actual_roi = measure_roi_in_ab_test()
```

---

**6. 业务应用场景不同**:

```python
响应率建模适用于:
✅ 预测客户流失（Churn Prediction）
✅ 推荐系统（谁会喜欢这个商品）
✅ 信用评分（谁会违约）

响应率建模不适用于:
❌ 营销干预（发券、广告投放）
❌ 个性化定价
❌ 政策效果评估

Uplift 建模适用于:
✅ 优惠券分发
✅ 广告定向
✅ 挽留（Retention）干预
✅ 个性化推荐（考虑干预效果）
✅ 临床试验（药物效果）
```

---

**7. 真实案例对比**:

```python
某电商平台的惨痛教训:

阶段 1: 使用响应率模型
模型: P(购买 | 用户特征)
策略: 给 top 20% 高购买概率用户发券

结果:
- 券使用率: 85%（看起来很好！）
- 但增量转化率: 仅 2%
- ROI: -60%（亏损）

原因:
发券的用户中:
- 80% 是 Sure Things（本来就会买）
- 15% 是 Persuadables
- 5% 是 Sleeping Dogs

→ 大部分券都浪费在 Sure Things 上

阶段 2: 切换到 Uplift 模型
模型: Uplift = P(购买|券,X) - P(购买|无券,X)
策略: 给 top 20% 高 Uplift 用户发券

结果:
- 券使用率: 65%（下降了）
- 但增量转化率: 18%（大幅提升）
- ROI: +25%（盈利！）

发券的用户中:
- 10% 是 Sure Things
- 85% 是 Persuadables
- 5% 是 Lost Causes

→ 精准定向 Persuadables

业务影响:
- 同样的预算，收入提升 40%
- 或者用 60% 的预算达到同样效果（节省 40%）
```

---

**常见误区**:

```python
误区 1: "响应率高 = 干预效果好"
❌ 错误
高响应率可能是因为用户本来就会买（Sure Things）

正确:
高 Uplift = 干预效果好

---

误区 2: "我们没有实验数据，所以只能用响应率模型"
❌ 错误
没有实验数据，响应率模型也无法估计因果效应

正确:
要做营销决策，必须先做实验获取 Uplift 数据
或者用 PSM/DML 等方法从观测数据估计（但要非常小心假设）

---

误区 3: "Uplift 模型太复杂，响应率模型够用了"
❌ 错误
复杂度的提升换来的是巨大的 ROI 提升

正确:
投资 Uplift 建模的回报远超成本
（见上面案例：ROI 从 -60% 到 +25%）
```

---

**实施建议**:

```python
如果你的公司还在用响应率模型:

步骤 1: 意识觉醒
- 向管理层展示 Sure Things 问题
- 计算当前策略的浪费程度

步骤 2: 小规模 Pilot
- 在一个细分市场做 Uplift 实验
- 对比响应率 vs Uplift 的 ROI

步骤 3: 逐步扩大
- 成功后推广到更多场景
- 建立 Uplift 建模平台

步骤 4: 持续迭代
- 定期 A/B 测试验证模型
- 更新训练数据
```

**关键洞察**:
> **响应率模型回答"谁会买"，Uplift 模型回答"因为你的营销谁会买"。做营销决策时，后者才是正确的问题。不要让"高响应率"的假象蒙蔽了你——那可能只是在给本来就会买的人发券罢了。**

---

In [ ]:
# 思考题 2: 为什么 "Sleeping Dogs" 用户发券反而转化率下降？举一个实际例子。

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: 在真实业务中，如何验证 Uplift 模型的准确性?

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 如果预算有限 (比如只能发券给 20% 的用户)，应该如何调整发券策略?

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: Uplift 建模相比传统的 "响应率建模" 有什么本质区别?

answer_5 = """
你的答案:



"""
print(answer_5)

## 📊 总结

### 用户分群策略

| 用户类型 | Uplift | 策略 | 业务价值 |
|----------|--------|------|----------|
| Persuadables | 高正 | 重点发券 | 💰 高 ROI |
| Sure Things | ~0 | 不发券 | 💸 节省成本 |
| Lost Causes | ~0 | 不发券 | 💸 节省成本 |
| Sleeping Dogs | 负 | 绝不发券 | ⚠️ 避免损失 |

### Uplift 建模 vs 响应率建模

| 方面 | 响应率建模 | Uplift 建模 |
|------|------------|-------------|
| 目标 | 预测谁会转化 | 预测谁会被影响 |
| 结果 | P(Y=1\|X) | P(Y=1\|T=1,X) - P(Y=1\|T=0,X) |
| 问题 | 找到 Sure Things | 找到 Persuadables |

### ROI 优化公式

$$\text{ROI} = \frac{N \times \text{Uplift} \times \text{Revenue} - N \times \text{Cost}}{N \times \text{Cost}}$$

In [ ]:
print("🎫 恭喜完成智能发券优化练习!")
print("\n你已经学会了:")
print("  ✓ 四类用户的概念和识别")
print("  ✓ T-Learner Uplift 模型")
print("  ✓ 用户分群策略")
print("  ✓ ROI 计算和策略对比")
print("\n下一步: A/B 测试增强 - CUPED!")